In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory
import os

base_dir = 'Imagens/'

# Diretórios para treino, validação e teste
train_dir = os.path.join(base_dir, 'train/train1/')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


In [3]:
IMG_SIZE = 150
BATCH_SIZE = 32

# Configuração do gerador de imagens sem data augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    
)

validation_dataset = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    
)

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    
)

Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [4]:
from tensorflow import keras
from tensorflow.keras import layers, models

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Construa o modelo
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)

x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)

x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)

x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)

x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)  

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

2024-05-25 19:31:15.696385: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-25 19:31:15.722676: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
# Treinar Rede
model.load_weights('Primeiro_treino.h5')
history = model.fit(train_dataset, epochs=50, validation_data=validation_dataset)

Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.9783 - loss: 0.0685

/home/tex/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 93s 295ms/step - accuracy: 0.9783 - loss: 0.0685 - val_accuracy: 0.5917 - val_loss: 3.8113
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 91s 288ms/step - accuracy: 0.9891 - loss: 0.0330 - val_accuracy: 0.6033 - val_loss: 3.8288
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 287ms/step - accuracy: 0.9897 - loss: 0.0315 - val_accuracy: 0.5782 - val_loss: 4.3744
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 286ms/step - accuracy: 0.9928 - loss: 0.0249 - val_accuracy: 0.5761 - val_loss: 4.1851
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 287ms/step - accuracy: 0.9820 - loss: 0.0712 - val_accuracy: 0.5892 - val_loss: 3.5818
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 286ms/step - accuracy: 0.9944 - loss: 0.0164 - val_accuracy: 0.5940 - val_loss: 3.8864
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 286ms/step - accuracy: 0.9989 - loss: 0.0043 - val_accuracy: 0.6044 - val_loss: 4.1234
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 286ms/step - accuracy: 1.0000 - loss: 3.9251e-04 -

KeyboardInterrupt: 

In [7]:
#Saving the model
model.save('Primeiro_treino.h5')

In [8]:
from tensorflow import keras

model = keras.models.load_model('Primeiro_treino.h5')
# Validacao da Rede
val_loss, val_acc = model.evaluate(validation_dataset)
print('val_acc:', val_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 91ms/step - accuracy: 0.6103 - loss: 4.7254
val_acc: 0.611299991607666


In [11]:
import matplotlib.pyplot as plt


# Extraindo os valores de acurácia e perda do histórico de treinamento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Definindo o número de épocas
epochs = range(1, len(acc) + 1)

# Plotando a acurácia de treinamento e validação
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

# Exibindo o gráfico de acurácia
plt.figure()

# Plotando a perda de treinamento e validação
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

# Exibindo o gráfico de perda
plt.show()


NameError: name 'history' is not defined